## Using PyQuickSQL Query Loader
Before importing quicksql you can change the default cache, either in your system's environment variables or like below.
The default is `tempfile.gettempdir()`.

In [ ]:
import os 
os.environ['QQ_CACHE_DIR']='path/to/cachedir'

In your sql file you must specify the start of a query with `-- name: queryname`, and the parameters within the query using `:paramname`.
You can optionally end a query with `-- :end`.

Now we can import quicksql and load up some queries.

In [1]:
import quicksql, os
queries = quicksql.LoadSQL(os.path.join(os.getcwd(),'example.sql'))
queries

LoadSQL(C:\Users\charl\PycharmProjects\PyQuickSQL\example.sql)
Query Name: contributing_employees, Params: num_orders, order_avg
Query Name: customer_orders, Params: product_id, sdate, edate

In [2]:
print(str(queries))

Queries from:: C:\Users\charl\PycharmProjects\PyQuickSQL\example.sql

-- Query:: contributing_employees
SELECT
  EmployeeID,
  COUNT(OrderID) AS NumberOfOrders,
  AVG(TotalAmount) AS AverageOrderAmount
FROM
  Orders
GROUP BY
  EmployeeID
HAVING
  COUNT(OrderID) > :num_orders AND AVG(TotalAmount) > :order_avg
ORDER BY
  AverageOrderAmount DESC;
/* This query selects the EmployeeID, counts the number of orders, and calculates the average order amount from an Orders table.
   It groups the results by EmployeeID, and includes only those employees who have more than :num_orders orders and where the average order amount is greater than $:order_avg.
   It orders the results by the average order amount in descending order. */

-- Query:: customer_orders
SELECT
  c.CustomerName,
  o.OrderDate,
  o.Status,
  (SELECT SUM(od.Quantity * od.UnitPrice) FROM OrderDetails od WHERE od.OrderID = o.OrderID) AS TotalValue
FROM
  Customers c
INNER JOIN Orders o ON c.CustomerID = o.CustomerID
WHERE
  o.Order

And lastly we can produce a query given the arguments specified above.

In [3]:
print(queries.contributing_employees(num_orders=5,order_avg=1000)+'\n\n')
print(queries.customer_orders(product_id=10,sdate='1-10-2022',edate=quicksql.NoStr("DATE'4-11-2023'"),something_not_a_param='test')+'\n\n')
print(queries.contributing_employees(num_orders=6)+'\n\n')

SELECT
  EmployeeID,
  COUNT(OrderID) AS NumberOfOrders,
  AVG(TotalAmount) AS AverageOrderAmount
FROM
  Orders
GROUP BY
  EmployeeID
HAVING
  COUNT(OrderID) > 5 AND AVG(TotalAmount) > 1000
ORDER BY
  AverageOrderAmount DESC;
/* This query selects the EmployeeID, counts the number of orders, and calculates the average order amount from an Orders table.
   It groups the results by EmployeeID, and includes only those employees who have more than 5 orders and where the average order amount is greater than $1000.
   It orders the results by the average order amount in descending order. */


Unused variables: something_not_a_param in query customer_orders
SELECT
  c.CustomerName,
  o.OrderDate,
  o.Status,
  (SELECT SUM(od.Quantity * od.UnitPrice) FROM OrderDetails od WHERE od.OrderID = o.OrderID) AS TotalValue
FROM
  Customers c
INNER JOIN Orders o ON c.CustomerID = o.CustomerID
WHERE
  o.OrderDate BETWEEN '1-10-2022' AND DATE'4-11-2023'
  AND EXISTS (SELECT 1 FROM OrderDetails od WHERE od

ValueError: Missing value for variable order_avg

We can see that the first one returns the query string, the second returns while printing a warning notifying of a variable not part of the query, and the third raises an error with a missing parameter.
For now only unordered but not optional `**kwargs` are supported. If it is worth implementing `*args` in the future, it will be done.
The parameters support any primitive type that can be converted into a string and collections of primitives eg tuples lists

## Using PyQuickSQL's Session Transient Asset Cache
This is a decorator designed specifically
This works like functools's cache but with some differences.